# Getting started quickly with Gate Set Tomography

The `pygsti` package provides multiple levels of abstraction over the core Gate Set Tomography (GST) algorithms.  This initial tutorial will show you how to work with `pygsti`'s highest level of abstraction to get you started using GST quickly.  Subsequent tutorials will delve into the details of `pygsti` objects and algorithms, and how to use them in detail.

## The `do_long_sequence_gst` driver function
Let's first look at how to use the  `do_long_sequence_gst` which combines all the steps of running typical GST algortithms into a single function. 

In [1]:
#Make print statements compatible with Python 2 and 3
from __future__ import print_function

In [2]:
#Import the pygsti module (always do this)
import pygsti

#### First, we need to specify what our desired gate set is, referred to as the "target gateset".
Gate sets and other `pygsti` objects are constructed using routines within `pygsti.construction`, and so we construct a gateset by calling `pygsti.construction.build_gateset`:

In [3]:
#Construct a target gateset
gs_target = pygsti.construction.build_gateset([2],[('Q0',)], ['Gi','Gx','Gy'], 
                                             [ "I(Q0)","X(pi/2,Q0)", "Y(pi/2,Q0)"],
                                             prepLabels=['rho0'], prepExpressions=["0"],
                                             effectLabels=['E0'], effectExpressions=["1"], 
                                             spamdefs={'plus': ('rho0','E0'), 'minus': ('rho0','remainder') } )

The parameters to `build_gateset`, specify:
 - The state space is dimension 2 (i.e. the density matrix is 2x2)
 
 
 - interpret this 2-dimensional space as that of a single qubit labeled "Q0" (label must begin with 'Q')
 
 
 - there are three gates: Idle, $\pi/2$ x-rotation, $\pi/2$ y-rotation
 
 
 - there is one state prep operation, which prepares the 0-state (the first basis element of the 2D state space)
 
 
 - there is one POVM (~ measurement) that projects onto the 1-state (the second basis element of the 2D state space)
 
 
 - the name of the state-prep then measure our POVM is `plus`
 
 
 - the name of the state-prep then measure something other than our POVM is `minus` 

Reading from and writing to files is done mostly via routines in `pygsti.io`. To store this gateset in a file (for reference or to load it somewhere else), you just call `pygsti.io.write_gateset`:

In [4]:
#Write it to a file
pygsti.io.write_gateset(gs_target, "tutorial_files/MyTargetGateset.txt")

#To load the gateset back into a python object, do:
# gs_target = pygsti.io.load_gateset("tutorial_files/MyTargetGateset.txt")

#### Next, we need to create fiducial, germ, and max-length lists:
These three lists will specify what experiments GST will use in its estimation procedure, and depend on the target gateset as well as the expected quality of the qubit being measured.  They are:

- fiducial gate strings (``fiducials``): gate sequences that immediately follow state preparation or immediately precede measurement.


- germ gate strings (``germs``): gate sequences that are repeated to produce a string that is as close to some "maximum length" as possible without exceeding it.


- maximum lengths (`maxLengths`): a list of maximum lengths used to specify the increasingly long gate sequences (via more germ repeats) used by each iteration of the GST estimation procedure.

To make GST most effective, these gate strings lists should be computed.  Typically this computation is done by the Sandia GST folks and the gate string lists are sent to you, though there is preliminary support within `pygsti` for computing these string lists directly.  Here, we'll assume we have been given the lists.  The maximum lengths list typically starts with [0,1] and then contains successive powers of two.  The largest maximum length should roughly correspond to the number of gates ones qubit can perform before becoming depolarized beyond ones ability to measure anything other than the maximally mixed state.  Since we're constructing gate string lists, the routines used are in `pygsti.construction`:

In [5]:
#Create fiducial gate string lists
fiducials = pygsti.construction.gatestring_list( [ (), ('Gx',), ('Gy',), ('Gx','Gx'), ('Gx','Gx','Gx'), ('Gy','Gy','Gy') ])

#Create germ gate string lists
germs = pygsti.construction.gatestring_list( [('Gx',), ('Gy',), ('Gi',), ('Gx', 'Gy',),
         ('Gx', 'Gy', 'Gi',), ('Gx', 'Gi', 'Gy',), ('Gx', 'Gi', 'Gi',), ('Gy', 'Gi', 'Gi',),
         ('Gx', 'Gx', 'Gi', 'Gy',), ('Gx', 'Gy', 'Gy', 'Gi',),
         ('Gx', 'Gx', 'Gy', 'Gx', 'Gy', 'Gy',)] )

#Create maximum lengths list
maxLengths = [0,1,2,4,8,16,32]

If we want to, we can save these lists in files (but this is not necessary):

In [6]:
pygsti.io.write_gatestring_list("tutorial_files/MyFiducials.txt", fiducials, "My fiducial gate strings")
pygsti.io.write_gatestring_list("tutorial_files/MyGerms.txt", germs, "My germ gate strings")

import pickle
pickle.dump( maxLengths, open("tutorial_files/MyMaxLengths.pkl", "wb"))

# To load these back into python lists, do:
#fiducials = pygsti.io.load_gatestring_list("tutorial_files/MyFiducials.txt")
#germs = pygsti.io.load_gatestring_list("tutorial_files/MyGerms.txt")
#maxLengths = pickle.load( open("tutorial_files/MyMaxLengths.pkl"))

#### Third, we generate (since we can't actually take) data and save a dataset
Before experimental data is obtained, it is useful to create a "template" dataset file which specifies which gate sequences are required to run GST.  Since we don't actually have an experiment for this example, we'll generate some "fake" experimental data from a set of gates that are just depolarized versions of the targets.  First we construct the list of experiments used by GST using `make_lsgst_experiment_list`, and use the result to specify which experiments to simulate.  The abbreviation "LSGST" (lowercase in function names to follow Python naming conventions) stands for "Long Sequence Gate Set Tomography", and refers to the more powerful flavor of GST that utilizes long sequences to find gate set estimates.  LSGST can be compared to Linear GST, or "LGST", which only uses short sequences and as a result provides much less accurate estimates.

In [7]:
#Create a list of GST experiments for this gateset, with
#the specified fiducials, germs, and maximum lengths
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths)

#Create an empty dataset file, which stores the list of experiments
#plus extra columns where data can be inserted
pygsti.io.write_empty_dataset("tutorial_files/MyDataTemplate.txt", listOfExperiments,
                              "## Columns = plus count, count total")

Since we don't actually have a experiment to generate real data, let's now create and save a dataset using depolarized target gates and spam operations:

In [8]:
#Create a gateset of depolarized gates and SPAM relative to target, and generate fake data using this gateset.
gs_datagen = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=1000000,
                                            sampleError="binomial", seed=2015)

We could at this point just use the generated dataset directly, but let's save it as though it were a file filled with experimental results.

In [9]:
#Save our dataset
pygsti.io.write_dataset("tutorial_files/MyDataset.txt", ds)

#Note; to load the dataset back again, do:
#ds = pygsti.io.load_dataset("tutorial_files/MyDataset.txt")

#### Fourth, we call the Analysis function
Now we're all set to call the driver routine.  All of the possible arguments to this function are detailed in the included help (docstring), and so here we just make a few remarks:
- For many of the arguments, you can supply either a filename or a python object (e.g. dataset, target gateset, gate string lists).


- `fiducials` is supplied twice since the state preparation fiducials (those sequences following a state prep) need not be the same as the measurement fiducials (those sequences preceding a measurement).


- Typically we want to constrain the resulting gates to be trace-preserving, so we leave `constrainToTP` set to `True` (the default).


- `gaugeOptRatio` specifies the ratio of the state preparation and measurement (SPAM) weight to the gate weight when performing a gauge optimization.  When this is set to 0.001, as below, the gate parameters are weighted 1000 times more relative to the SPAM parameters.  Typically it is good to weight the gates parameters more heavily since GST amplifies gate parameter errors via long gate sequences but cannot amplify SPAM parameter errors.  If unsure, 0.001 is a good value to start with.

In [10]:
results = pygsti.do_long_sequence_gst("tutorial_files/MyDataset.txt", gs_target, 
                                        fiducials, fiducials, germs, maxLengths,
                                        gaugeOptRatio=1e-3, constrainToTP=True)

Loading tutorial_files/MyDataset.txt: 100%
    
--- LGST ---
    ('LGST: Singular values of I_tilde (truncating to first 4 of 6) = \n', array([  4.24408562e+00,   1.16716561e+00,   9.46738951e-01,
         9.42572736e-01,   1.23055187e-03,   1.11706986e-03]))
      LGST DEBUG: Gi before trunc to first 4 row and cols = 

   4.2438   0.0021   0.0005  -0.0019        0        0
   0.0063   1.0499  -0.0006   0.0006   0.0003  -0.0002
   0.0016   0.0007   0.8525   0.0024        0  -0.0003
  -0.0080  -0.0005   0.0011   0.8485   0.0010  -0.0003
  -0.0004   0.0011   0.0013   0.0018  -0.0006  -0.0003
   0.0017  -0.0003  -0.0002  -0.0007   0.0015  -0.0002

      LGST DEBUG: Gx before trunc to first 4 row and cols = 

   4.2416   0.0074   0.0259  -0.0159        0        0
   0.1360   0.0026  -0.8219   0.4668        0        0
  -0.0041   0.8328   0.2001   0.3521   0.0011        0
  -0.0066  -0.4484   0.3730   0.6484   0.0003   0.0010
   0.0002        0   0.0011   0.0008  -0.0006  -0.0006
   0.0012 

In [11]:
import pickle
s = pickle.dumps(results)
r2 = pickle.loads(s)
print(r2.gatesets['final estimate'])

rho0 =    0.7071        0   0.0002   0.7068


E0 =    0.7071        0        0  -0.7059


Gi = 
   1.0000        0        0        0
        0   0.9000  -0.0002        0
        0        0   0.9000        0
        0  -0.0002        0   0.9000


Gx = 
   1.0000        0        0        0
        0   0.9001        0        0
        0        0        0  -0.9000
        0        0   0.9000  -0.0002


Gy = 
   1.0000        0        0        0
        0        0        0   0.9000
        0        0   0.9000   0.0001
        0  -0.9000        0   0.0001





The analysis routine returns a `pygsti.report.Results` object which encapsulates intermediate and final GST estimates, as well as quantities derived from these "raw" estimates.  (The object also caches derived quantities so that repeated queries for the same quanties do not require recalculation.)  Finally, a `Results` object can generate reports and presentations containing many of the raw and derived GST results.  We give examples of these uses below. 

In [12]:
# Access to raw GST best gateset estimate
print(results.gatesets['final estimate'])

rho0 =    0.7071        0   0.0002   0.7068


E0 =    0.7071        0        0  -0.7059


Gi = 
   1.0000        0        0        0
        0   0.9000  -0.0002        0
        0        0   0.9000        0
        0  -0.0002        0   0.9000


Gx = 
   1.0000        0        0        0
        0   0.9001        0        0
        0        0        0  -0.9000
        0        0   0.9000  -0.0002


Gy = 
   1.0000        0        0        0
        0        0        0   0.9000
        0        0   0.9000   0.0001
        0  -0.9000        0   0.0001





In [13]:
#create a full GST report (most detailed and pedagogical; best for those getting familiar with GST)
results.create_full_report_pdf(confidenceLevel=95, filename="tutorial_files/easy_full.pdf", verbosity=2)

      
  --- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
   40s           0.0008806033
   54s           0.0008721466
   70s           0.0008713328
    The resulting min sqrt(sum(gateCIs**2)): 0.000871333
  *** Generating tables ***
   Iter 00 of 16 :   Generating table: targetSpamTable (w/95% CIs)
   Iter 01 of 16 :   Generating table: targetGatesTable (w/95% CIs)
   Iter 02 of 16 :   Generating table: datasetOverviewTable (w/95% CIs)
   Iter 03 of 16 :   Generating table: bestGatesetSpamTable (w/95% CIs)
   Iter 04 of 16 :   Generating table: bestGatesetSpamParametersTable (w/95% CIs)
   Iter 05 of 16 :   Generating table: bestGatesetGatesTable (w/95% CIs)
   Iter 06 of 16 :   Generating table: bestGatesetChoiTable (w/95% CIs)
   Iter 07 of 16 :   Generating table: bestGatesetDecompTable (w/95% CIs)
   Iter 08 of 16 :   Generating table: bestGatesetRotnAxisTable (w/95% CIs)
   Iter 09 of 16 :   Generating table: bestGatesetClosestUnitaryTable (w/95% CIs)


/usr/local/lib/python3.5/site-packages/numpy/lib/twodim_base.py:233: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m = zeros((N, M), dtype=dtype)
/usr/local/lib/python3.5/site-packages/numpy/lib/twodim_base.py:240: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m[:M-k].flat[i::M+1] = 1


   Iter 10 of 16 :   Generating table: bestGatesetVsTargetTable (w/95% CIs)
   Iter 11 of 16 :   Generating table: bestGatesetErrorGenTable (w/95% CIs)
   Iter 12 of 16 :   Generating table: fiducialListTable (w/95% CIs)
   Iter 13 of 16 :   Generating table: prepStrListTable (w/95% CIs)
   Iter 14 of 16 :   Generating table: effectStrListTable (w/95% CIs)
   Iter 15 of 16 :   Generating table: germListTable (w/95% CIs)
   Iter 16 of 16 :   Generating table: progressTable (w/95% CIs)
  *** Generating plots ***
  LogL plots (2): 
   Iter 0 of 1 :   Generating figure: bestEstimateColorBoxPlot (w/95% CIs)
  Generating figure: bestEstimateColorBoxPlot
   Iter 1 of 1 :   Generating figure: invertedBestEstimateColorBoxPlot (w/95% CIs)
  Generating figure: invertedBestEstimateColorBoxPlot
  
  *** Merging into template file ***
  Latex file(s) successfully generated.  Attempting to compile with pdflatex...
  Initial output PDF tutorial_files/easy_full.pdf successfully generated.
  Final outpu

In [14]:
#create a brief GST report (just highlights of full report but fast to generate; best for folks familiar with GST)
results.create_brief_report_pdf(confidenceLevel=95, filename="tutorial_files/easy_brief.pdf", verbosity=2)

  *** Generating tables ***
  Retrieving cached table: bestGatesetSpamTable (w/95% CIs)
  Retrieving cached table: bestGatesetSpamParametersTable (w/95% CIs)
  Retrieving cached table: bestGatesetGatesTable (w/95% CIs)
  Retrieving cached table: bestGatesetDecompTable (w/95% CIs)
  Retrieving cached table: bestGatesetRotnAxisTable (w/95% CIs)
  Retrieving cached table: bestGatesetVsTargetTable (w/95% CIs)
  Retrieving cached table: bestGatesetErrorGenTable (w/95% CIs)
  Retrieving cached table: progressTable (w/95% CIs)
  *** Generating plots ***
  *** Merging into template file ***
  Latex file(s) successfully generated.  Attempting to compile with pdflatex...
  Initial output PDF tutorial_files/easy_brief.pdf successfully generated.
  Final output PDF tutorial_files/easy_brief.pdf successfully generated. Cleaning up .aux and .log files.


In [15]:
#create GST slides (tables and figures of full report in latex-generated slides; best for folks familiar with GST)
results.create_presentation_pdf(confidenceLevel=95, filename="tutorial_files/easy_slides.pdf", verbosity=2)

  *** Generating tables ***
  Retrieving cached table: targetSpamTable (w/95% CIs)
  Retrieving cached table: targetGatesTable (w/95% CIs)
  Retrieving cached table: datasetOverviewTable (w/95% CIs)
  Retrieving cached table: bestGatesetSpamTable (w/95% CIs)
  Retrieving cached table: bestGatesetSpamParametersTable (w/95% CIs)
  Retrieving cached table: bestGatesetGatesTable (w/95% CIs)
  Retrieving cached table: bestGatesetChoiTable (w/95% CIs)
  Retrieving cached table: bestGatesetDecompTable (w/95% CIs)
  Retrieving cached table: bestGatesetRotnAxisTable (w/95% CIs)
  Retrieving cached table: bestGatesetVsTargetTable (w/95% CIs)
  Retrieving cached table: bestGatesetErrorGenTable (w/95% CIs)
  Retrieving cached table: fiducialListTable (w/95% CIs)
  Retrieving cached table: prepStrListTable (w/95% CIs)
  Retrieving cached table: effectStrListTable (w/95% CIs)
  Retrieving cached table: germListTable (w/95% CIs)
  Retrieving cached table: progressTable (w/95% CIs)
  *** Generating pl

In [16]:
#create GST slides (tables and figures of full report in Powerpoint slides; best for folks familiar with GST)
#Requires python-pptx and ImageMagick to be installed
results.create_presentation_ppt(confidenceLevel=95, filename="tutorial_files/easy_slides.pptx", verbosity=2)

  *** Generating tables ***
   Iter 00 of 15 :   Retrieving cached table: targetSpamTable (w/95% CIs)
   Iter 01 of 15 :   Retrieving cached table: targetGatesTable (w/95% CIs)
   Iter 02 of 15 :   Retrieving cached table: datasetOverviewTable (w/95% CIs)
   Iter 03 of 15 :   Retrieving cached table: bestGatesetSpamTable (w/95% CIs)
   Iter 04 of 15 :   Retrieving cached table: bestGatesetSpamParametersTable (w/95% CIs)
   Iter 05 of 15 :   Retrieving cached table: bestGatesetGatesTable (w/95% CIs)
   Iter 06 of 15 :   Retrieving cached table: bestGatesetChoiTable (w/95% CIs)
   Iter 07 of 15 :   Retrieving cached table: bestGatesetDecompTable (w/95% CIs)
   Iter 08 of 15 :   Retrieving cached table: bestGatesetRotnAxisTable (w/95% CIs)
   Iter 09 of 15 :   Retrieving cached table: bestGatesetVsTargetTable (w/95% CIs)
   Iter 10 of 15 :   Retrieving cached table: bestGatesetErrorGenTable (w/95% CIs)
   Iter 11 of 15 :   Retrieving cached table: fiducialListTable (w/95% CIs)
   Iter 12 

If all has gone well, the above lines have produced the four primary types of reports `pygsti` is capable of generating:
- A "full" report, [tutorial_files/easy_full.pdf](tutorial_files/easy_full.pdf). This is the most detailed and pedagogical of the reports, and is best for those getting familiar with GST.


- A "brief" report, [tutorial_files/easy_brief.pdf](tutorial_files/easy_brief.pdf). This Contains just the highlights of the full report but much faster to generate, and is best for folks familiar with GST.


- PDF slides, [tutorial_files/easy_slides.pdf](tutorial_files/easy_slides.pdf). These slides contain tables and figures of the full report in LaTeX-generated (via `beamer`) slides, and is best for folks familiar with GST who want to show other people their great results.


- PPT slides, [tutorial_files/easy_slides.pptx](tutorial_files/easy_slides.pptx). These slides contain the same information as PDF slides, but in MS Powerpoint format.  These slides won't look as nice as the PDF ones, but can be used for merciless copying and pasting into your other Powerpoint presentations... :)

## Speeding things up by using Standard constructions
A significant component of running GST as show above is constructing things: the target gateset, the fiducial, germ, and maximum-length lists, etc.  We've found that many people who use GST have one of only a few different target gatesets, and for these commonly used target gatesets we've created modules that perform most of the constructions for you.  If you gateset isn't one of these standard ones then you'll have to follow the above approach for now, but please let us know and we'll try to add a module for your gateset in the future.

The standard construction modules are located under `pygsti.construction` (surprise, surprise) and are prefixed with "`std`".  In the example above, our gateset (comprised of single qubit $I$, X($\pi/2$), and Y($\pi/2$) gates) is one of the commonly used gatesets, and relevant constructions are importable via:

In [17]:
#Import the "stardard 1-qubit quantities for a gateset with X(pi/2), Y(pi/2), and idle gates"
from pygsti.construction import std1Q_XYI

We follow the same order of constructing things as above, but it's much easier since almost everything has been constructed already:

In [18]:
gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [0,1,2,4,8,16,32] #still need to define this manually

We generate a fake dataset as before:

In [19]:
gs_datagen = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=1000000,
                                            sampleError="binomial", seed=1234)

And run the analysis function (this time using the dataset object directly instead of loading from a file), and then create a report in the specified file.

In [20]:
results = pygsti.do_long_sequence_gst(ds, gs_target, fiducials, fiducials, germs, maxLengths)
results.create_full_report_pdf(confidenceLevel=95,filename="tutorial_files/MyEvenEasierReport.pdf",verbosity=2)

    
--- LGST ---
    ('LGST: Singular values of I_tilde (truncating to first 4 of 6) = \n', array([  4.24409906e+00,   1.16704331e+00,   9.46915249e-01,
         9.43164143e-01,   2.28752927e-03,   6.68710597e-04]))
      LGST DEBUG: Gi before trunc to first 4 row and cols = 

   4.2437   0.0021        0  -0.0019        0        0
   0.0082   1.0503        0   0.0005  -0.0014   0.0012
  -0.0002  -0.0006   0.8530  -0.0005   0.0008        0
  -0.0101   0.0010   0.0001   0.8482  -0.0012  -0.0009
  -0.0008   0.0012  -0.0002        0  -0.0003   0.0006
  -0.0008   0.0007  -0.0009   0.0009  -0.0002  -0.0006

      LGST DEBUG: Gx before trunc to first 4 row and cols = 

   4.2416   0.0072  -0.0199  -0.0232        0        0
   0.1371   0.0026   0.6409   0.6936        0        0
   0.0062  -0.6525   0.4540  -0.4164   0.0008   0.0003
  -0.0048  -0.6836  -0.4330   0.3950  -0.0004        0
  -0.0011        0  -0.0002   0.0003  -0.0026  -0.0002
  -0.0017  -0.0001        0   0.0009  -0.0006  -0.000

/usr/local/lib/python3.5/site-packages/numpy/lib/twodim_base.py:233: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m = zeros((N, M), dtype=dtype)
/usr/local/lib/python3.5/site-packages/numpy/lib/twodim_base.py:240: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m[:M-k].flat[i::M+1] = 1


   Iter 10 of 16 :   Generating table: bestGatesetVsTargetTable (w/95% CIs)
   Iter 11 of 16 :   Generating table: bestGatesetErrorGenTable (w/95% CIs)
   Iter 12 of 16 :   Generating table: fiducialListTable (w/95% CIs)
   Iter 13 of 16 :   Generating table: prepStrListTable (w/95% CIs)
   Iter 14 of 16 :   Generating table: effectStrListTable (w/95% CIs)
   Iter 15 of 16 :   Generating table: germListTable (w/95% CIs)
   Iter 16 of 16 :   Generating table: progressTable (w/95% CIs)
  *** Generating plots ***
  LogL plots (2): 
   Iter 0 of 1 :   Generating figure: bestEstimateColorBoxPlot (w/95% CIs)
  Generating figure: bestEstimateColorBoxPlot
   Iter 1 of 1 :   Generating figure: invertedBestEstimateColorBoxPlot (w/95% CIs)
  Generating figure: invertedBestEstimateColorBoxPlot
  
  *** Merging into template file ***
  Latex file(s) successfully generated.  Attempting to compile with pdflatex...
  Initial output PDF tutorial_files/MyEvenEasierReport.pdf successfully generated.
  Fi

Now open [tutorial_files/MyEvenEasierReport.pdf](tutorial_files/MyEvenEasierReport.pdf) to see the results.  You've just run GST (again)!

In [21]:
# Printing a Results object gives you information about how to extract information from it
print(results)

----------------------------------------------------------
---------------- pyGSTi Results Object -------------------
----------------------------------------------------------

I can create reports for you directly, via my create_XXX
functions, or you can query me for result data via members:

 .dataset    -- the DataSet used to generate these results

 .gatesets   -- a dictionary of GateSet objects w/keys:
 ---------------------------------------------------------
  iteration estimates
  iteration estimates pre gauge opt
  final estimate
  seed
  target

 .gatestring_lists   -- a dict of GateString lists w/keys:
 ---------------------------------------------------------
  iteration
  final
  prep fiducials
  all
  germs
  effect fiducials

 .tables   -- a dict of ReportTable objects w/keys:
 ---------------------------------------------------------
  blankTable
  targetSpamTable
  targetSpamBriefTable
  targetGatesTable
  datasetOverviewTable
  fiducialListTable
  prepStrListTable
  